In [1]:
import pandas as pd
import re
#make pandas print more of the content of a dataframe
pd.set_option('display.max_colwidth', None)

In [2]:
#save a new csv
df = pd.read_csv(r"..\data\entities.csv")

In [3]:
#first turn all triplets into a list, currently they  are a string of a list
df['triplets'] = df['triplets'].apply(lambda x: eval(x))
triplets_list = df.triplets.to_list()
#flatten the list of lists
triplets_list = [item for sublist in triplets_list for item in sublist]
def standardize_entry(entry):
    if isinstance(entry, dict):
        return (entry['head'], entry['type'], entry['tail'])
    elif isinstance(entry, tuple) and len(entry) == 3:
        return entry
    else:
        raise ValueError(f"Unexpected entry format: {entry}")


# Now, reprocess your data. As the function may return lists of lists now, we need to flatten it
standardized_data = list(map(standardize_entry,triplets_list))
df = pd.DataFrame(standardized_data, columns=['head', 'type', 'tail'])

df = df.drop_duplicates()


In [4]:
def build_mapping_dict(df, normalize_func):
    mapping_dict = {}
    for column in df.columns:
        original_series = df[column].copy()  # copy the original series
        normalized_series = df[column].apply(normalize_func)  # normalize the text in the column
        mapping_dict[column] = dict(zip(original_series, normalized_series))  # create a mapping from original to normalized text
    return mapping_dict

def normalize_text(text):
    normalized_text = text.lower() # lowercase text
    normalized_text = re.sub(r'\s+', '_', normalized_text) # replace all whitespace with underscores
    normalized_text = re.sub(r'[^a-zA-Z0-9_]', '', normalized_text) #remove all non alphanumeric characters
    normalized_text = re.sub(r'[.,!@#$%^&*(){}<>?:;\[\]-]', '', normalized_text) #remove all punctuation
    return normalized_text

mapping_dict = build_mapping_dict(df, normalize_text)

# Now you can apply the normalization to the dataframe
for column in df.columns:
    df[column] = df[column].apply(normalize_text)

# And save the mapping dictionary
import json
with open(r"D:\repos\anonymous_phenomena_timeline\data\mapping_dict.json", 'w') as f:
    json.dump(mapping_dict, f)


In [ ]:

for column in df.columns:
    df[column] = df[column].apply(normalize_text)
#save to csv
df.to_csv(r"D:\repos\project_amanita\amanita_export_working_area\anon_pdf\graph_data_final.csv", index=False)